In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Depression"
cohort = "GSE135524"

# Input paths
in_trait_dir = "../../input/GEO/Depression"
in_cohort_dir = "../../input/GEO/Depression/GSE135524"

# Output paths
out_data_file = "../../output/preprocess/Depression/GSE135524.csv"
out_gene_data_file = "../../output/preprocess/Depression/gene_data/GSE135524.csv"
out_clinical_data_file = "../../output/preprocess/Depression/clinical_data/GSE135524.csv"
json_path = "../../output/preprocess/Depression/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I'll write code for the current step that fixes the syntax issues and correctly processes the clinical data.

```python
# 1. Gene Expression Data Availability
# Checking if gene expression data is available based on the background information
# The study seems to involve gene expression analysis in whole blood samples
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# For trait (Depression) - using psychomotor composite score as a proxy for depression severity
trait_row = 7  # psychomotor composite score

# For age
age_row = 1  # age

# For gender
gender_row = 2  # Sex

# 2.2 Data Type Conversion functions
def convert_trait(value):
    """Convert psychomotor composite score to a continuous value."""
    if value is None or not isinstance(value, str):
        return None
    # Extract the value after the colon
    parts = value.split(':', 1)
    if len(parts) < 2:
        return None
    try:
        # Convert to float as it's a continuous measure
        return float(parts[1].strip())
    except (ValueError, TypeError):
        return None

def convert_age(value):
    """Convert age to a continuous value."""
    if value is None or not isinstance(value, str):
        return None
    # Extract the value after the colon
    parts = value.split(':', 1)
    if len(parts) < 2:
        return None
    try:
        # Convert to int as age is typically a whole number
        return int(parts[1].strip())
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender to a binary value (0 for female, 1 for male)."""
    if value is None or not isinstance(value, str):
        return None
    # Extract the value after the colon
    parts = value.split(':', 1)
    if len(parts) < 2:
        return None
    
    gender_value = parts[1].strip().lower()
    if 'female' in gender_value:
        return 0
    elif 'male' in gender_value:
        return 1
    return None

# 3. Save Metadata
# Determine trait data availability based on trait_row
is_trait_available = trait_row is not None

# Save cohort info using the provided function
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Only proceed if trait_row is not None
if trait_row is not None:
    # Create DataFrame from sample characteristics dictionary
    # We'll use the sample characteristics dictionary from the previous step output
    sample_characteristics_dict = {
        0: ['individual: Subject 71', 'individual: Subject 73', 'individual: Subject 74', 'individual: Subject 11', 
            'individual: Subject 95', 'individual: Subject 41', 'individual: Subject 91', 'individual: Subject 108', 
            'individual: Subject 109', 'individual: Subject 86', 'individual: Subject 66', 'individual: Subject 85', 
            'individual: Subject 54', 'individual: Subject 20', 'individual: Subject 103', 'individual: Subject 15', 
            'individual: Subject 13', 'individual: Subject 46', 'individual: Subject 30', 'individual: Subject 24', 
            'individual: Subject 105', 'individual: Subject 48', 'individual: Subject 31', 'individual: Subject 37', 
            'individual: Subject 104', 'individual: Subject 114', 'individual: Subject 61', 'individual: Subject 19', 
            'individual: Subject 113', 'individual: Subject 110'],
        1: ['age: 55', 'age: 56', 'age: 24', 'age: 61', 'age: 40', 'age: 35', 'age: 41', 'age: 26', 'age: 64', 
            'age: 43', 'age: 29', 'age: 34', 'age: 25', 'age: 28', 'age: 27', 'age: 33', 'age: 51', 'age: 52', 
            'age: 45', 'age: 42', 'age: 46', 'age: 49', 'age: 38', 'age: 39', 'age: 30', 'age: 32', 'age: 36', 
            'age: 22', 'age: 37', 'age: 44'],
        2: ['Sex: Male', 'Sex: Female'],
        3: ['bmi: 24.079', 'bmi: 28.313', 'bmi: 26.549', 'bmi: 25.833', 'bmi: 40.436', 'bmi: 21.658', 'bmi: 52.288', 
            'bmi: 43.208', 'bmi: 34.132', 'bmi: 32.029', 'bmi: 32.058', 'bmi: 31.462', 'bmi: 27.924', 'bmi: 44.688', 
            'bmi: 38.714', 'bmi: 47.837', 'bmi: 37.67', 'bmi: 27.179', 'bmi: 29.966', 'bmi: 35.833', 'bmi: 24.355', 
            'bmi: 48.598', 'bmi: 30.613', 'bmi: 27.653', 'bmi: 28.954', 'bmi: 33.489', 'bmi: 27.836', 'bmi: 39.585', 
            'bmi: 30.08', 'bmi: 21.696'],
        4: ['race: White', 'race: Black'],
        5: ['hamd score excluding item 8: 25', 'hamd score excluding item 8: 23', 'hamd score excluding item 8: 21', 
            'hamd score excluding item 8: 28', 'hamd score excluding item 8: 24', 'hamd score excluding item 8: 29', 
            'hamd score excluding item 8: 22', 'hamd score excluding item 8: 19', 'hamd score excluding item 8: 30', 
            'hamd score excluding item 8: 20', 'hamd score excluding item 8: 26', 'hamd score excluding item 8: 27', 
            'hamd score excluding item 8: 17', 'hamd score excluding item 8: 18'],
        6: ['college (yes/no): 1', 'college (yes/no): 0'],
        7: ['psychomotor composite score: -0.06308', 'psychomotor composite score: 1.30018', 
            'psychomotor composite score: -0.41699', 'psychomotor composite score: -0.17858', 
            'psychomotor composite score: -0.64284', 'psychomotor composite score: -0.32956', 
            'psychomotor composite score: 0.00218', 'psychomotor composite score: 2.51701', 
            'psychomotor composite score: 1.2821', 'psychomotor composite score: 0.18995', 
            'psychomotor composite score: -0.40531', 'psychomotor composite score: 0.57381', 
            'psychomotor composite score: 0.8438', 'psychomotor composite score: 0.15064', 
            'psychomotor composite score: 1.89099', 'psychomotor composite score: 1.03181', 
            'psychomotor composite score: -1.73217', 'psychomotor composite score: -0.65832', 
            'psychomotor composite score: 0.78252', 'psychomotor composite score: -1.36392', 
            'psych


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I'll provide a properly formatted Python code solution for this step:

```python
# Let's analyze the dataset by looking at the results of the previous step
import os
import pandas as pd
import numpy as np
import json
import gzip
from typing import Callable, Optional, Dict, Any
import re

# Let's look for the required files in the cohort directory
files = os.listdir(in_cohort_dir)
print(f"Files in {in_cohort_dir}:")
for file in files:
    print(file)

# First, let's check if there's a series matrix file which would contain the gene expression data
series_matrix_files = [f for f in files if "series_matrix" in f.lower()]
sample_characteristics = {}

if series_matrix_files:
    print(f"Found series matrix file: {series_matrix_files[0]}")
    matrix_file = os.path.join(in_cohort_dir, series_matrix_files[0])
    
    # Use gzip to open the compressed file
    with gzip.open(matrix_file, 'rt') as f:
        # Read header lines to analyze the dataset
        header_lines = []
        line = f.readline()
        while line and not line.startswith('!series_matrix_table_begin'):
            header_lines.append(line.strip())
            line = f.readline()
        
        # Extract sample characteristics
        sample_chars_lines = [line for line in header_lines if "!Sample_characteristics_ch1" in line]
        
        if sample_chars_lines:
            print("Sample characteristics found in series matrix file (first 10):")
            for i, line in enumerate(sample_chars_lines[:10]):
                print(line.strip())
            
            # Build a dictionary to store unique values for each characteristic row
            for i, line in enumerate(sample_chars_lines):
                # Get sample characteristic values
                values = re.findall(r'"([^"]*)"', line)
                if i not in sample_characteristics:
                    sample_characteristics[i] = []
                sample_characteristics[i].extend(values)
    
    # Print unique values for each characteristic row to help identify relevant rows
    print("\nUnique values for each characteristic row:")
    for row, values in sample_characteristics.items():
        unique_values = set(values)
        print(f"Row {row}: {unique_values}")
    
    # Check if platform description indicates gene expression data
    platform_lines = [line for line in header_lines if "!Platform_" in line]
    platform_tech_lines = [line for line in platform_lines if "_technology" in line.lower() or "_type" in line.lower()]
    
    print("\nPlatform information:")
    for line in platform_tech_lines:
        print(line)

# Based on our analysis of the characteristics rows, let's determine:
# 1. Gene expression data availability
# Assuming it's gene expression data based on platform information
is_gene_available = True  # Update this based on actual platform information

# 2. Trait, age, and gender data availability and row identification
# For now using None as placeholders, we'll update these after analyzing the data
trait_row = None  # Row containing depression status
age_row = None    # Row containing age information
gender_row = None # Row containing gender information

# Let's analyze the sample characteristics to determine the correct rows
for row, values in sample_characteristics.items():
    unique_values = set(values)
    lower_values = [v.lower() if isinstance(v, str) else v for v in unique_values]
    
    # Check for trait information (depression)
    if any(("depress" in v or "mdd" in v or "control" in v or "patient" in v or "case" in v or 
            "healthy" in v or "disorder" in v or "illness" in v) for v in lower_values if isinstance(v, str)):
        if len(unique_values) > 1:  # Ensure it's not a constant feature
            trait_row = row
            print(f"Found Depression trait information at row {row}: {unique_values}")
    
    # Check for age information
    if any(("age" in v.lower() or "years" in v.lower()) for v in lower_values if isinstance(v, str)):
        if len(unique_values) > 1:  # Ensure it's not a constant feature
            age_row = row
            print(f"Found age information at row {row}: {unique_values}")
    
    # Check for gender information
    if any(("gender" in v.lower() or "sex" in v.lower() or "female" in v.lower() or "male" in v.lower()) 
           for v in lower_values if isinstance(v, str)):
        if len(unique_values) > 1:  # Ensure it's not a constant feature
            gender_row = row
            print(f"Found gender information at row {row}: {unique_values}")

# Define conversion functions based on the actual data patterns
def convert_trait(value):
    """Convert depression trait value to binary (0: control, 1: case)."""
    if not isinstance(value, str):
        return None
    
    value = value.lower().strip()
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if any(term in value for term in ["control", "healthy", "non-depressed", "normal", "nd"]):
        return 0
    elif any(term in value for term in ["depressed", "mdd", "depression", "patient", "case"]):
        return 1
    else:
        return None

def convert_age(value):
    """Convert age value to continuous numeric value."""
    if not isinstance(value, str):
        return None
    
    # Extract numeric part after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Try to extract numbers from the string
    matches = re.findall(r'\d+(?:\.\d+)?', value)
    if matches:
        try:
            age = float(matches[0])
            return age
        except ValueError:
            return None
    return None

def convert_gender(value):
    """Convert gender value to binary (0: female, 1: male)."""
    if not isinstance(value, str):
        return None
    
    value = value.lower().strip()
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if any(term in value for term in ["female", "f", "woman", "women"]):
        return 0
    elif any(term in value for term in ["male", "m", "man", "men"]):
        return 1
    else:
        return None

# Save metadata with initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# If clinical data is available, extract and save it
if trait_row is not None:
    # Create a DataFrame from the sample characteristics
    sample_ids = []
    
    with gzip.open(os.path.join(in_cohort_dir, series_matrix_files[0]), 'rt') as f:
        for line in f:
            if line.startswith('!Sample_geo_accession'):
                sample_ids = re.findall(r'"([^"]*)"', line)
                break
    
    # Create a DataFrame with the correct structure for geo_select_clinical_features
    clinical_data = pd.DataFrame(index=range(len(sample_characteristics)), columns=sample_ids)
    
    for row, values in sample_characteristics.items():
        for i, value in enumerate(values):
            if i < len(sample_ids):
                clinical_data.iloc[row, i] = value
    
    # Extract clinical features
    selected_clinical = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row


### Step 4: Gene Data Extraction

In [ ]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 5: Gene Identifier Review

In [ ]:
# Analyzing the gene identifiers

# Based on the observed identifiers (ILMN_XXXXXXX), these are Illumina probe IDs,
# not standard human gene symbols. Illumina probe IDs are platform-specific identifiers
# used in Illumina microarray platforms.

# These identifiers need to be mapped to standard human gene symbols
# for consistent cross-study analysis and biological interpretation.

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# Check if there are any platforms defined in the SOFT file that might contain annotation data
with gzip.open(soft_file, 'rt') as f:
    soft_content = f.read()

# Look for platform sections in the SOFT file
platform_sections = re.findall(r'^!Platform_title\s*=\s*(.+)$', soft_content, re.MULTILINE)
if platform_sections:
    print(f"Platform title found: {platform_sections[0]}")

# Try to extract more annotation data by reading directly from the SOFT file
# Look for lines that might contain gene symbol mappings
symbol_pattern = re.compile(r'ID_REF\s+Symbol|ID\s+Gene Symbol', re.IGNORECASE)
annotation_lines = []
with gzip.open(soft_file, 'rt') as f:
    for line in f:
        if symbol_pattern.search(line):
            annotation_lines.append(line)
            # Collect the next few lines to see the annotation structure
            for _ in range(10):
                annotation_lines.append(next(f, ''))

if annotation_lines:
    print("Found potential gene symbol mappings:")
    for line in annotation_lines:
        print(line.strip())

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("\nGene annotation preview:")
print(preview_df(gene_annotation, n=10))

# If we need an alternative source of mapping, check if there are any other annotation files in the cohort directory
cohort_files = os.listdir(in_cohort_dir)
annotation_files = [f for f in cohort_files if 'annotation' in f.lower() or 'platform' in f.lower()]
if annotation_files:
    print("\nAdditional annotation files found in the cohort directory:")
    for file in annotation_files:
        print(file)


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Determine which columns in the gene annotation DataFrame contain probe IDs and gene symbols
# Based on the preview, 'ID' column contains probe IDs like ILMN_XXXXXXX, matching the gene_data index
# 'Symbol' column contains gene symbols

# 2. Extract the probe-to-gene mapping columns
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')
print(f"Mapping dataframe shape: {mapping_df.shape}")
print("First few rows of the mapping dataframe:")
print(mapping_df.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few gene symbols:")
print(gene_data.index[:10])

# 4. Normalize gene symbols to ensure consistency
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene expression data shape after normalization: {gene_data.shape}")
print("First few normalized gene symbols:")
print(gene_data.index[:10])

# Save the gene expression data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. We'll normalize gene symbols in the gene expression data
# Note: We've already done this in step 7, so we can skip this part

# 2. Link the clinical and genetic data
# First, let's re-acquire the clinical data from the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Load the normalized gene expression data saved in the previous step
gene_data = pd.read_csv(out_gene_data_file, index_col=0)
print(f"Loaded gene expression data shape: {gene_data.shape}")

# Define conversion functions for clinical features
def convert_trait(value):
    """Convert psychomotor composite score to continuous value."""
    if isinstance(value, str) and ":" in value:
        parts = value.split(":", 1)
        if len(parts) == 2:
            try:
                return float(parts[1].strip())
            except ValueError:
                return None
    return None

def convert_age(value):
    """Convert age to a continuous value."""
    if isinstance(value, str) and ":" in value:
        parts = value.split(":", 1)
        if len(parts) == 2:
            try:
                return int(parts[1].strip())
            except ValueError:
                return None
    return None

def convert_gender(value):
    """Convert gender to a binary value (0 for female, 1 for male)."""
    if isinstance(value, str) and ":" in value:
        parts = value.split(":", 1)
        if len(parts) == 2:
            gender_str = parts[1].strip().lower()
            if "female" in gender_str:
                return 0
            elif "male" in gender_str:
                return 1
    return None

# Extract clinical features using the correct rows from the sample characteristics
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=7,  # Psychomotor composite score (row 7)
    convert_trait=convert_trait,
    age_row=1,    # Age (row 1)
    convert_age=convert_age,
    gender_row=2, # Sex (row 2)
    convert_gender=convert_gender
)

print(f"Selected clinical data shape: {selected_clinical_df.shape}")
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save clinical data for future reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5] if not linked_data.empty else "Linked data is empty")

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Check for bias in features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from whole blood samples of depressed patients with psychomotor slowing traits."
)

# 6. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")